In [1]:
import os
import glob
import json
import dask.bag
from kerchunk.netCDF3 import NetCDF3ToZarr
from kerchunk.combine import MultiZarrToZarr
import numpy as np
import xarray as xr

In [2]:
atm_files = sorted(glob.glob("/projects/0/nwo2021025/archive/mres_b.e10.B2000_CAM5.f05_t12.001/atm/hist/*.h0.*.nc"))
print(f"{len(atm_files)=}: ({atm_files[0].split('.')[-2]} - {atm_files[-1].split('.')[-2]})")

len(atm_files)=648: (2045-01 - 2098-12)


In [3]:
def open_mfdataset(filepaths, ncstore_dir='~/kerchunk', **kwargs):
    """a faster alternative to xr.open_mfdataset using kerchunk
    
    This function uses kerchunk to create an NC_STORE reference file,
    that instructs the program how to read the netCDF files efficiently. 
    Coordinates must be consistent throughout all files.
    The NC_STORE is saved after first use, and will be read on each 
    subsequent usage of this function.
    
    Parameters:
    filepaths : Str or List[Str]
        (list of) netCDF file names, may contain wild cards
    ncstore_dir: Pathlike
        Path where NC_STORE reference files will be saved
    kwargs: dict
        any additional keyword arguments are passed on to xr.open_dataset
        
    Returns: xr.Dataset
        a Dataset instance containing all the netCDF data
        
    v0.0
    """
    
    # make sorted list of absolute filepaths
    if isinstance(filepaths, str):
        filepaths = glob.glob(filepaths)
    filepaths = sorted([os.path.abspath(fp) for fp in filepaths])
    if len(filepaths) == 1: # use xr.open_dataset directly if there is one file
        return xr.open_dataset(filepaths, **kwargs)
    
    # set default keyword arguments for xr.open_dataset on NC_STORE file
    default_kw = {'engine':'kerchunk', 'storage_options':{'target_protocol':'file'}}
    for (k,v) in default_kw.items():
        if k in kwargs:
            print(f'open_mfdataset(): ignoring keyword {k}')
        kwargs[k] = v
    
    # create NC_STORE filename from netCDF filename, including timestamp
    # of first and last file. Open and return dataset if the file already exists
    ncstore_dir = os.path.expanduser(ncstore_dir)
    timestr = lambda i: os.path.basename(filepaths[i]).split('.')[-2] # timestamp
    ncstorefile = (os.path.basename(filepaths[0])
                   .replace(timestr(0),f"{timestr(0)}_{timestr(-1)}")
                   .replace('.nc','.json'))
    ncstore_path = os.path.join(ncstore_dir, ncstorefile)
    if not os.path.exists(ncstore_dir):
        os.mkdir(ncstore_dir)
    elif os.path.exists(ncstore_path):
        print(f"Reading combined kerchunk reference file {ncstore_path}")
        return xr.open_dataset(ncstore_path, **kwargs)
    
    # make new NC_STORE data
    filebag = dask.bag.from_sequence(filepaths, npartitions=None)
    reffiles = (filebag.map(NetCDF3ToZarr, inline_threshold=0, max_chunk_size=0)
                .map(lambda z: z.translate()).compute())
    mzz = MultiZarrToZarr(reffiles, concat_dims=['time'], coo_map={'time':'cf:time'})
    
    # write NC_STORE data and return opened dataset
    with open(f"{ncstore_path}", "wb") as f:
       print(f"Writing combined kerchunk reference file {ncstore_path}")
       f.write(json.dumps(mzz.translate()).encode())
    
    return xr.open_dataset(ncstore_path, **kwargs)

In [4]:
%%time
ds = open_mfdataset("/projects/0/nwo2021025/archive/mres_b.e10.B2000_CAM5.f05_t12.001/atm/hist/*.h0.*.nc", chunks={'time':1})
ds

Reading combined kerchunk reference file /home/jasperdj/kerchunk/mres_b.e10.B2000_CAM5.f05_t12.001.cam2.h0.2045-01_2098-12.json
CPU times: user 752 ms, sys: 102 ms, total: 854 ms
Wall time: 865 ms


<xarray.Dataset> Size: 2TB
Dimensions:          (time: 648, lev: 30, lat: 384, lon: 576, ilev: 31,
                      slat: 383, slon: 576, nbnd: 2)
Coordinates:
  * ilev             (ilev) float64 248B 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * lat              (lat) float64 3kB -90.0 -89.53 -89.06 ... 89.06 89.53 90.0
  * lev              (lev) float64 240B 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * lon              (lon) float64 5kB 0.0 0.625 1.25 ... 358.1 358.8 359.4
  * slat             (slat) float64 3kB -89.77 -89.3 -88.83 ... 88.83 89.3 89.77
  * slon             (slon) float64 5kB -0.3125 0.3125 0.9375 ... 358.4 359.1
  * time             (time) object 5kB 2045-02-01 00:00:00 ... 2099-01-01 00:...
Dimensions without coordinates: nbnd
Data variables: (12/247)
    ABSORB           (time, lev, lat, lon) float32 17GB dask.array<chunksize=(1, 30, 384, 576), meta=np.ndarray>
    AEROD_v          (time, lat, lon) float32 573MB dask.array<chunksize=(1, 384, 576), meta=np.ndarray>
    ANRAIN           (time, lev, lat, lon) float32 17GB dask.array<chunksize=(1, 30, 384, 576), meta=np.ndarray>
    ANSNOW           (time, lev, lat, lon) float32 17GB dask.array<chunksize=(1, 30, 384, 576), meta=np.ndarray>
    AODABS           (time, lat, lon) float32 573MB dask.array<chunksize=(1, 384, 576), meta=np.ndarray>
    AODDUST1         (time, lat, lon) float32 573MB dask.array<chunksize=(1, 384, 576), meta=np.ndarray>
    ...               ...
    soa_a2_SRF       (time, lat, lon) float32 573MB dask.array<chunksize=(1, 384, 576), meta=np.ndarray>
    sol_tsi          (time) float64 5kB dask.array<chunksize=(1,), meta=np.ndarray>
    time_bnds        (time, nbnd) object 10kB dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_written     (time) |S8 5kB dask.array<chunksize=(1,), meta=np.ndarray>
    w_stag           (time, slat) float64 2MB dask.array<chunksize=(1, 383), meta=np.ndarray>
    wnummax          (time, lat) int32 995kB dask.array<chunksize=(1, 384), meta=np.ndarray>
Attributes:
    Conventions:      CF-1.0
    Version:          $Name$
    case:             mres_b.e10.B2000_CAM5.f05_t12.001
    host:             tcn896.local.sne
    initial_file:     rcp8.5_co2_f05_t12.cam2.i.2045-01-01-00000.nc
    logname:          renew1
    revision_Id:      $Id$
    source:           CAM
    title:            UNSET
    topography_file:  /projects/0/nwo2021025/cesm1_0_4/inputdata/atm/cam/topo...